In [52]:
import os
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import dominate
from selenium.webdriver.remote.webelement import WebElement as Element
del os.environ['DISPLAY']


KeyError: 'DISPLAY'

In [2]:
from pathlib import Path

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument("start-maximized")
options.add_argument("--headless")
options.add_argument("--remote-debugging-port=9222")
driver = webdriver.Chrome(options=options)

In [61]:
driver.get("http://ec2-3-131-244-37.us-east-2.compute.amazonaws.com:7770/")
driver.implicitly_wait(10) # seconds

In [59]:
JS_BUILD_CSS_SELECTOR = ("for(var e=arguments[0],n=[],i=function(e,n){if(!e||!n)return 0;f"
    "or(var i=0,a=e.length;a>i;i++)if(-1==n.indexOf(e[i]))return 0;re"
    "turn 1};e&&1==e.nodeType&&'HTML'!=e.nodeName;e=e.parentNode){if("
    "e.id){n.unshift('#'+e.id);break}for(var a=1,r=1,o=e.localName,l="
    "e.className&&e.className.trim().split(/[\\s,]+/g),t=e.previousSi"
    "bling;t;t=t.previousSibling)10!=t.nodeType&&t.nodeName==e.nodeNa"
    "me&&(i(l,t.className)&&(l=null),r=0,++a);for(var t=e.nextSibling"
    ";t;t=t.nextSibling)t.nodeName==e.nodeName&&(i(l,t.className)&&(l"
    "=null),r=0);n.unshift(r?o:o+(l?'.'+l.join('.'):':nth-child('+a+'"
    ")'))}return n.join(' > ');")



In [156]:
import dominate.tags


clickables = {}

def set_attribute(element: Element, attribute, value):
    driver.execute_script("arguments[0].setAttribute(arguments[1], arguments[2]);", element, attribute, value)

def register_clickable(element: Element, name: str):
    clickables[name] = element
    set_attribute(element, 'data-clickable-id', name)

def common(node: dominate.tags.html_tag, element: Element, recipe, name = ''):
    if 'name' in recipe and recipe['name']:
        node['name'] = (name + "." if name else '') + recipe['name']
    if 'clickable' in recipe and recipe['clickable']:
        if 'name' not in recipe:
            raise Exception('Clickable element must have name')
        register_clickable(element, (name + "." if name else '') + recipe['name'])
    for key in ['alt', 'src', 'href', 'title', 'type', 'value']:
        value = element.get_dom_attribute(key)
        if value:
            node[key] = value
        
    return node

actions = {}

def get_text(element: Element, recipe, name = ''):
    elementText = element.text # sometime NOT work
    if not elementText:
        elementText = element.get_attribute("innerText")
    if not elementText:
        elementText = element.get_attribute("textContent")
    node = getattr(dominate.tags, element.tag_name)(elementText)
    node = common(node, element, recipe, name)
    if 'name' in recipe and recipe['name']:
        name = (name + "." if name else '') + recipe['name']
    if 'radio' in recipe and recipe['radio']:
        if element.get_attribute('checked'):
            node.text += ' (selected)'
    return node

def strip_add_children(element: Element, recipe, name = ''):
    tag = element.tag_name
    node = getattr(dominate.tags, tag)()
    node = common(node, element, recipe, name)
    if 'name' in recipe and recipe['name']:
        name = (name + "." if name else '') + recipe['name']
    with node:
        for child in recipe['children']:
            elements = element.find_elements(By.CSS_SELECTOR, child['selector'])
            for child_element in elements:
                actions[child['action']](child_element, child, name)
    return node


actions = {
    'get_text': get_text,
    'strip_add_children': strip_add_children
}

In [157]:
recipe = {
    'selector': 'html',
    'action': 'strip_add_children',
    'clickable': False,
    'children': [
        {
            'selector': 'head',
            'name': '',
            'action': 'strip_add_children',
            'children': [
                {
                    'selector': 'title',
                    'action': 'get_text',
                }
            ]
        },
        {
            'selector': 'nav',
            'name': 'nav',
            'action': 'strip_add_children',
            'children': [
                {
                    'selector': 'ul',
                    'action': 'strip_add_children',
                    'children': [
                        {
                            'selector': 'li',
                            'action': 'get_text',
                            'clickable': True,
                        }
                    ]
                }
            ]
        }
    ]
}

In [158]:
print(strip_add_children(driver.find_element(By.CSS_SELECTOR, recipe['selector']), recipe))

<html>
  <head>
    <title>One Stop Market</title>
  </head>
  <nav name="nav"></nav>
</html>


In [153]:
driver.find_element(By.CSS_SELECTOR, 'html').find_element(By.CSS_SELECTOR, 'nav')

<selenium.webdriver.remote.webelement.WebElement (session="fb40a98df1e22d9246fdf35d537a2abf", element="f.61335BC334089FFDCEA69597859FA8DD.d.C47B2B45524AC20193F32EA937697B61.e.157")>

In [136]:
title = driver.find_element(By.CSS_SELECTOR, 'html').find_element(By.CSS_SELECTOR, 'head').find_element(By.CSS_SELECTOR, 'title')

In [63]:
title.get_attribute('innerHTML')

'One Stop Market'

In [64]:
type(title)

selenium.webdriver.remote.webelement.WebElement

In [65]:
driver.execute_script(JS_BUILD_CSS_SELECTOR, title)

'head > title'

In [77]:
btn = driver.find_element(By.CSS_SELECTOR, '#maincontent > div.columns > div > div:nth-child(3) > div > div.block-content > div.products-grid.grid > ol > li:nth-child(3) > div > div > div.product-item-inner > div > div.actions-primary > form > button')


In [79]:
btn.setProperty('data-clickable-id', '123')

AttributeError: 'WebElement' object has no attribute 'setProperty'

In [83]:
btn.get_attribute('data-clickable-id')

'123'

In [92]:
btn.get_attribute('src')

In [98]:
title = dominate.tags.title('title')

In [114]:
print(title)

<title checked="True">title (selected)
  <div>test</div>
</title>


In [111]:
title.text += ' (selected)'

In [113]:
with title:
    dominate.tags.div('test')